In [1]:
import collections
import copy
import cspUtil
import dataUtil

In [71]:
def countSequences(inputList, N):
    out = collections.defaultdict(float)
    for i in range(len(inputList)-N+1):
        if [inputList[i+k] != None for k in range(N)] == [True for _ in range(N)]:
            seq = tuple([inputList[i+k] for k in range(N)])
            out[seq] += 1
    print out

    factor= 1.0 / sum(out.itervalues())
    for k in out:
          out[k] *= 10*factor

    return out

In [39]:
midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/chor_armure.mid"
windowStart, windowEnd = 10, 15
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()
rythmUnit, windowedPrimTrack = mypattern.getCorrupt(windowStart, windowEnd)

Loaded /Users/robin/Desktop/CS221_projet/Open_MIDI_file/chor_armure.mid


In [119]:
def createRythmCSP(windowedPrimTrack, bi=True, tri=False):
    cspRythm = cspUtil.CSP()

    # Creating "units" variables
    transitions = [note[1] for note in windowedPrimTrack]
    for unitNum in range(windowStart, windowEnd+1):
        unit = windowedPrimTrack[unitNum]
        cspRythm.add_variable(('U', unitNum), [(unit[0],True),(unit[0],False)])

    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(transitions, 2)
        #print "bi"
        #print biSequences
        for varNum in range(windowStart, windowEnd):
            cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1), lambda u1, u2 : biSequences[(u1[1], u2[1])])
        # Add corner cases
        cspRythm.add_unary_factor(('U', windowStart), lambda u : biSequences[(windowedPrimTrack[windowStart-1][1], u[1])])
        cspRythm.add_unary_factor(('U', windowEnd)  , lambda u : biSequences[(u[1], windowedPrimTrack[windowEnd+1][1])])

    if tri:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(windowStart, windowEnd+1):
            cspRythm.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(transitions, 3)

        for varNum in range(windowStart, windowEnd+1):
            # Add constraint B_i[0] = U_i[1]
            cspRythm.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != windowEnd:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspRythm.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspRythm.add_unary_factor(('B', windowStart), lambda b: b[1] == windowedPrimTrack[windowStart-1])
        cspRythm.add_unary_factor(('B', windowEnd)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[windowEnd+1])] \
                        * triSequences[(b[0], windowedPrimTrack[windowEnd+1], windowedPrimTrack[windowEnd+2])])

    return cspRythm

In [120]:
def createPitchCSP(windowedPitchPrimTrack, bi=True, tri=False):
    cspPitch = cspUtil.CSP()
    pitchTrack = [note[0] for note in windowedPrimTrack]
    # Creating "units" variables #
    # Domains
    minPitch, maxPitch = pitchTrack[0], pitchTrack[0]
    for pitch in pitchTrack:
        if type(pitch) == int:
            minPitch, maxPitch = min(minPitch, pitch), max(maxPitch, pitch)
    domains = tuple(list(range(minPitch, maxPitch+1)) + ['silence'])
    
    for pitchNum in range(windowStart, windowEnd+1):
        cspPitch.add_variable(('U', pitchNum), copy.deepcopy(domains))

    # Create binary factor: must keep the same pitch if previous duration = True
    for varNum in range(windowStart, windowEnd): 
        if windowedPitchPrimTrack[varNum][1]:
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda x, y : x == y)
    
    # Create binary factors proportional to frequence of apparition of pairs
    biSequences = countSequences(pitchTrack, 2)
    #print "bi"
    #print biSequences
    for varNum in range(windowStart, windowEnd):
        cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda p1, p2 : biSequences[(p1, p2)])
    # Add corner cases
    cspPitch.add_unary_factor(('U', windowStart), lambda p : biSequences[(pitchTrack[windowStart-1], p)])
    cspPitch.add_unary_factor(('U', windowEnd)  , lambda p : biSequences[(p, pitchTrack[windowEnd+1])])

    """
    if triSeq:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(windowStart, windowEnd+1):
            cspPitch.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(windowedPrimTrack, 3)

        for varNum in range(windowStart, windowEnd+1):
            # Add constraint B_i[0] = U_i[1]
            cspPitch.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != windowEnd:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspPitch.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspPitch.add_unary_factor(('B', windowStart), lambda b: b[1] == windowedPrimTrack[windowStart-1])
        cspPitch.add_unary_factor(('B', windowEnd)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[windowEnd+1])] \
                        * triSequences[(b[0], windowedPrimTrack[windowEnd+1], windowedPrimTrack[windowEnd+2])])
        """

    return cspPitch

In [121]:
midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid"
windowStart, windowEnd = 10, 15
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()
rythmUnit, windowedPrimTrack = mypattern.getCorrupt(windowStart, windowEnd)

Loaded /Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid


In [123]:
CSP = createRythmCSP(windowedPrimTrack, True, False)
search_rythm = cspUtil.BacktrackingSearch()
search_rythm.solve(CSP)

windowedPitchPrimTrack = windowedPrimTrack[:]
for i, note in enumerate(windowedPitchPrimTrack[1:]):
    if note == (None, None):
        windowedPitchPrimTrack[i+1] = search_rythm.optimalAssignment[('U', i+1)]

defaultdict(<type 'float'>, {(False, True): 23.0, (True, False): 25.0, (False, False): 2.0, (True, True): 6.0})
Found 1 optimal assignments with weight 25317.171354 in 127 operations
First assignment took 7 operations
One of the optimal assignments is
('U', 10) (None, True)
('U', 11) (None, False)
('U', 12) (None, True)
('U', 13) (None, False)
('U', 14) (None, True)
('U', 15) (None, False)


In [125]:
CSP = createPitchCSP(windowedPitchPrimTrack, True, False)
search_pitch = cspUtil.BacktrackingSearch()
search_pitch.solve(CSP)

defaultdict(<type 'float'>, {(74, 72): 2.0, (77, 76): 1.0, (76, 76): 12.0, (77, 77): 3.0, (74, 74): 9.0, (76, 77): 2.0, (77, 79): 2.0, (72, 74): 2.0, (74, 76): 3.0, (72, 72): 10.0, (76, 74): 3.0, (79, 77): 1.0, (79, 79): 6.0})
Found 1 optimal assignments with weight 0.011256 in 29 operations
First assignment took 7 operations
One of the optimal assignments is
('U', 10) 77
('U', 11) 77
('U', 12) 76
('U', 13) 76
('U', 14) 74
('U', 15) 74
